<a href="https://colab.research.google.com/github/slayerzeroa/Railway_Accident_BERT/blob/main/Tagtog_Preprocess_Entity_Recognitioin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread
!pip install oauth2client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 라이브러리
import torch
from torch import nn, optim, tensor
from torch.utils.data import DataLoader, Dataset,TensorDataset, random_split
import torch.nn.functional as F

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, mean_squared_error

# from transformers import BertTokenizer

import json
import glob
import re
import os
import ast
import html
import requests

import zipfile
import shutil

import pandas as pd

from google.colab import files

In [ ]:
#로그인 위치
url = "https://tagtog.com/-login"

#다운로드 위치
file_url = 'https://www.tagtog.com/slayerzeroa/Railway_BERT/-downloads/dataset-as-anndoc' 
o_file = 'Railway_Annotation.zip'  
if os.path.exists(o_file):
    os.remove(o_file)

#로그인 정보
login_info = {
    'loginid' : 'slayerzeroa@gmail.com',
    'password' : 'gyysxw5rU3NzrYX',
}

#로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)

    with open(o_file,"wb") as output:
        output.write(r.content)

In [ ]:
#압축파일 풀기
folder_path = "./tagtog_relation_extraction/"

zip_ = zipfile.ZipFile("Railway_Annotation.zip")
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
zip_.extractall(folder_path)

In [ ]:
#폴더 경로
folder_name = "./tagtog_relation_extraction/Railway_BERT/"

#context list   # context == plain.html
context_name_list = os.listdir(folder_name + "plain.html/pool")
print(context_name_list)

#relation 폴더 경로   # relation == ann.json
relation_folder_paths = glob.glob(folder_name + "ann.json/master/pool")
#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/" + c for c in context_name_list]
print(contexts_folders_paths)

#anntation_lenged 정보
annotations_legend = folder_name + "annotations-legend.json"
with open(annotations_legend,"r") as f:
    annotations_legend = json.load(f)


['aPSRwvP5rtizz2O093I3oFX9uzjS-20180818.pdf.plain.html', 'aVuARZFRfgJMD0E67Coa.7G8Z9Pe-20200226.pdf.plain.html', 'arqduk0rnomRe20HKEeXlVPK8qx8-20191022.pdf.plain.html', 'awUyBdL4lPpVlBOPjSzMhAuRk81a-20181208.pdf.plain.html', 'a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf.plain.html', 'aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt.plain.html']
['./tagtog_relation_extraction/Railway_BERT/plain.html/pool/aPSRwvP5rtizz2O093I3oFX9uzjS-20180818.pdf.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/aVuARZFRfgJMD0E67Coa.7G8Z9Pe-20200226.pdf.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/arqduk0rnomRe20HKEeXlVPK8qx8-20191022.pdf.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/awUyBdL4lPpVlBOPjSzMhAuRk81a-20181208.pdf.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt.pla

In [ ]:
with open('./tagtog_relation_extraction/Railway_BERT/ann.json/master/pool/aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt.ann.json', "r", encoding="UTF-8") as f:
  test = json.load(f)

print(test)
print(test['entities'])

{'annotatable': {'parts': ['s1v1']}, 'anncomplete': False, 'sources': [], 'metas': {}, 'entities': [{'classId': 'e_28', 'part': 's1v1', 'offsets': [{'start': 4, 'text': '차량'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_6', 'part': 's1v1', 'offsets': [{'start': 19, 'text': '대차가'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_23', 'part': 's1v1', 'offsets': [{'start': 23, 'text': '선로'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_22', 'part': 's1v1', 'offsets': [{'start': 31, 'text': '탈선하였다'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}], 'relations': []}
[{'classId': 'e_28', 'part': 's1

In [ ]:
check_entity = dict()

for i in test['entities']:
  check_entity[i['offsets'][0]['text']] = i['classId']

print(check_entity)

{'차량': 'e_28', '대차가': 'e_6', '선로': 'e_23', '탈선하였다': 'e_22'}


In [ ]:
with open('/content/tagtog_relation_extraction/Railway_BERT/plain.html/pool/aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt.plain.html', "r") as f:
  sample = f.read()

print(sample)

<!DOCTYPE html >
<html id="aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt" data-origid="text.txt" class="anndoc" data-anndoc-version="3.6" lang="" xml:lang="" xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <meta charset="UTF-8"/>
    <meta name="generator" content="net.tagtog.anndoc.v3.parsers.general.VerbatimParser_v1_0_1"/>
    <title>aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt</title>
  </head>
  <body>
    <article>
      <section data-type="">
        <div class="content">
          <pre id="s1v1">5번째 차량 ( 3513호 ) 앞 대차가 선로 우측으로 탈선하였다.</pre>
        </div>
      </section>
    </article>
  </body>
</html>



In [ ]:
def remove_tag(html_text):
  check = html_text
  check = re.sub('<title>.*</title>', '', check)
  check = re.sub('(<([^>]+)>)', '', check).strip()
  check = re.sub('\n', '', check)
  check = re.sub(' +', ' ', check)
  check = re.sub('·','', check)
  check = re.sub('○', '', check)
  return check

In [ ]:
removing = remove_tag(sample)
print(removing)

5번째 차량 ( 3513호 ) 앞 대차가 선로 우측으로 탈선하였다.


In [ ]:
!pip install transformers
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
#punkt 다운로드
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
check = word_tokenize(removing)

In [ ]:
print(check)

['5번째', '차량', '(', '3513호', ')', '앞', '대차가', '선로', '우측으로', '탈선하였다', '.']


In [ ]:
checking_list = []

for i in check:
  if i in check_entity:
    checking_list.append(annotations_legend[check_entity[i]])
  if i not in check_entity:
    checking_list.append('O')

In [ ]:
print(checking_list)

['O', 'OBJECT', 'O', 'O', 'O', 'O', 'IMPLIED', 'SOLUTION', 'O', 'ACCIDENT', 'O']


In [ ]:
print(' '.join(checking_list))

O OBJECT O O O O IMPLIED SOLUTION O ACCIDENT O
